In [1]:
# Installing libraries 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from nltk.stem import PorterStemmer
from textblob import TextBlob # pip install textblob
import nltk
nltk.download('wordnet')
#import torch # conda install pytorch torchvision -c pytorch

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexisang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Reading in test.csv file into variable 'test'
test = pd.read_csv('C:/Users/Admin/OneDrive/Spring 19 Classes/STA 141C/Project/jigsaw-toxic-comment-classification-challenge/test.csv')

In [5]:
# Information about 'test' dataframe
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 2 columns):
id              153164 non-null object
comment_text    153164 non-null object
dtypes: object(2)
memory usage: 2.3+ MB


In [6]:
# Sample of 'test' dataframe
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [7]:
# Reading in test_labels.csv into variable 'test_labels'
test_labels = pd.read_csv('C:/Users/Admin/OneDrive/Spring 19 Classes/STA 141C/Project/jigsaw-toxic-comment-classification-challenge/test_labels.csv')

In [8]:
# Information about 'test_labels' dataframe
test_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 7 columns):
id               153164 non-null object
toxic            153164 non-null int64
severe_toxic     153164 non-null int64
obscene          153164 non-null int64
threat           153164 non-null int64
insult           153164 non-null int64
identity_hate    153164 non-null int64
dtypes: int64(6), object(1)
memory usage: 8.2+ MB


In [9]:
# Sample of 'test_labels' dataframe
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [10]:
# Reading in train.csv file into variable 'train'
train = pd.read_csv('C:/Users/Admin/OneDrive/Spring 19 Classes/STA 141C/Project/jigsaw-toxic-comment-classification-challenge/train.csv')

In [11]:
# Information about 'train' dataframe
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [12]:
# Sample of 'train' dataframe
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [13]:
# Dimensions of 'train' dataframe
# 159,571 rows and 8 columns (comment id, comment text, 6 toxicity classification labels)
train.shape

(159571, 8)

In [14]:
# Counting how many of each type of toxic comment there is
# 0 row indicates number of comments that do not fall under that specific toxicity classification
# 1 row indicates number of comments that do fall under that specific toxicity classification
train.apply(pd.value_counts, axis = 0).head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,NaN,NaN,144277.0,157976.0,151122.0,159093.0,151694.0,158166.0
1,NaN,NaN,15294.0,1595.0,8449.0,478.0,7877.0,1405.0
!\n\nI won't stop! LOCK IT UP A$$HOLE,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
"!\n\nIt is now inside your bones, your head. Haunt you forever",NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
"!\n\nSuck my DIHCK! Nihgga!\n\nYou don't want people to know the TRUTH about utorrant, you FAG",NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Sample of comment text that classifies as a 'toxic' comment
train.loc[train['toxic'] == 1]['comment_text'].head()

6          COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
12    Hey... what is it..\n@ | talk .\nWhat is it......
16    Bye! \n\nDon't look, come or think of comming ...
42    You are gay or antisemmitian? \n\nArchangel WH...
43             FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!
Name: comment_text, dtype: object

In [16]:
# Sample of comment text that classifies as an 'insult' comment
train.loc[train['insult'] == 1]['comment_text'].head()

6          COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
42    You are gay or antisemmitian? \n\nArchangel WH...
43             FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!
55    Stupid peace of shit stop deleting my stuff as...
56    =Tony Sidaway is obviously a fistfuckee. He lo...
Name: comment_text, dtype: object

# Feature extraction

## Number of words

In [17]:
## FEATURE EXTRACTION ##
### NUMBER OF WORDS ###

# Splits each comment based on whitespace and counts the number of words; adds results to 'word_count' column
train['word_count'] = train['comment_text'].apply(lambda x: len(str(x).split(" ")))
test['word_count'] = test['comment_text'].apply(lambda x: len(str(x).split(" ")))
# Sample of 'train' dataframe 'comment_text' column with new 'word_count' column
train[['comment_text','word_count']].head()

,comment_text,word_count
0,Explanation\nWhy the edits made under my usern...,42
1,D'aww! He matches this background colour I'm s...,18
2,"Hey man, I'm really not trying to edit war. It...",42
3,"""\nMore\nI can't make any real suggestions on ...",112
4,"You, sir, are my hero. Any chance you remember...",13


## Number of characters

In [18]:
### NUMBER OF CHARACTERS ###

# Counts number of characters in each comment
train['char_count'] = train['comment_text'].str.len() # Includes spaces
# Sample of 'train' dataframe 'comment_text' column with new 'char_count' column
train[['comment_text', 'char_count']].head()

,comment_text,char_count
0,Explanation\nWhy the edits made under my usern...,264
1,D'aww! He matches this background colour I'm s...,112
2,"Hey man, I'm really not trying to edit war. It...",233
3,"""\nMore\nI can't make any real suggestions on ...",622
4,"You, sir, are my hero. Any chance you remember...",67


In [19]:
# Applying character count code to 'test' dataframe
test['char_count'] = test['comment_text'].str.len() # Includes spaces

### Number of Uppercase words

In [20]:
### NUMBER OF UPPERCASE WORDS ###

# Counts the number of words in comment text in which all the characters in a word are all uppercase 
train['upper'] = train['comment_text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
# Sample of 'train' dataframe 'comment_text' column with new 'upper' column
train[['comment_text','upper']].head()

,comment_text,upper
0,Explanation\nWhy the edits made under my usern...,2
1,D'aww! He matches this background colour I'm s...,1
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",5
4,"You, sir, are my hero. Any chance you remember...",0


In [21]:
# Applying uppercase count code to 'test' dataframe
test['upper'] = test['comment_text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

# Basic Pre-processing

### Lowercase

In [22]:
## BASIC PRE-PROCESSING ##
### LOWERCASE ###

# Editing 'comment_text' so all characters are lower case and words are separated by a single whitespace
train['comment_text'] = train['comment_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
# Sample of 'train' dataframe 'comment_text' column with new lower case edits
train['comment_text'].head()

0    explanation why the edits made under my userna...
1    d'aww! he matches this background colour i'm s...
2    hey man, i'm really not trying to edit war. it...
3    " more i can't make any real suggestions on im...
4    you, sir, are my hero. any chance you remember...
Name: comment_text, dtype: object

In [23]:
# Applying lowercase edit code to 'test' dataframe
test['comment_text'] = test['comment_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

### Removing Punctuation

In [24]:
### REMOVING PUNCTUATION ###

# Editing 'comment_text' and using RegEx expression to remove non-alphanumeric characters and non-whitespace characters
train['comment_text'] = train['comment_text'].str.replace('[^\w\s]','')
# Sample of 'train' dataframe 'comment_text' column with new punctuation edits
train['comment_text'].head()                            

0    explanation why the edits made under my userna...
1    daww he matches this background colour im seem...
2    hey man im really not trying to edit war its j...
3     more i cant make any real suggestions on impr...
4    you sir are my hero any chance you remember wh...
Name: comment_text, dtype: object

In [25]:
# Applying punctuation edit code to 'test' dataframe
test['comment_text'] = test['comment_text'].str.replace('[^\w\s]','')

### Removal of Stop Words

In [26]:
### REMOVAL OF STOP WORDS ###

# Importing stop words list
from nltk.corpus import stopwords
# Assigning English stopwords to variable 'stop'
stop = stopwords.words('english')
# Editing 'comment_text' to remove stop words from variable 'stop' and separate words with single whitespace
train['comment_text'] = train['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
# Sample of 'train' dataframe 'comment_text' column with new stop word edits 
train['comment_text'].head()

0    explanation edits made username hardcore metal...
1    daww matches background colour im seemingly st...
2    hey man im really trying edit war guy constant...
3    cant make real suggestions improvement wondere...
4                  sir hero chance remember page thats
Name: comment_text, dtype: object

In [27]:
# Applying stop word edit code to 'test' dataframe
test['comment_text'] = test['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

### Common Word Removal

In [28]:
### COMMON WORD REMOVAL ###

# Counting number of times a word appears across all comments and selecting the top 10 most frequent words 
# (at the top of the list)
freq = pd.Series(' '.join(train['comment_text']).split()).value_counts()[:10]
freq

article      55403
page         45611
wikipedia    35557
talk         31288
please       29607
would        29212
one          28057
like         27705
dont         26102
see          21486
dtype: int64

In [29]:
# Editing 'comment_text' to remove common words from variable 'freq' and separate words with single whitespace
freq = list(freq.index)
train['comment_text'] = train['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# Sample of 'train' dataframe 'comment_text' column with common word edits 
train['comment_text'].head()

0    explanation edits made username hardcore metal...
1    daww matches background colour im seemingly st...
2    hey man im really trying edit war guy constant...
3    cant make real suggestions improvement wondere...
4                       sir hero chance remember thats
Name: comment_text, dtype: object

In [30]:
# Applying common word edit code to 'test' dataframe
freq = pd.Series(' '.join(test['comment_text']).split()).value_counts()[:10]
freq = list(freq.index)
test['comment_text'] = test['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

### Remove Rare Words

In [31]:
### REMOVE RARE WORDS ###

# Counting number of times a word appears across all comments and selecting the top 10 least frequent words 
# (at the bottom of the list)
freq = pd.Series(' '.join(train['comment_text']).split()).value_counts()[-10:]
freq

comicon                                                          1
httpwwwhousegovcorrinebrownccorner107cornersocialsecurityhtml    1
draftgodwin                                                      1
commercialmilitary                                               1
augenmusik                                                       1
klović                                                           1
writning                                                         1
handsmany                                                        1
wwwangelfirecomca3fresnocoptsindexhtml                           1
sunnybrook                                                       1
dtype: int64

In [32]:
# Editing 'comment_text' to remove rare words from variable 'freq' and separate words with single whitespace
freq = list(freq.index)
train['comment_text'] = train['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# Sample of 'train' dataframe 'comment_text' column with rare word edits 
train['comment_text'].head()

0    explanation edits made username hardcore metal...
1    daww matches background colour im seemingly st...
2    hey man im really trying edit war guy constant...
3    cant make real suggestions improvement wondere...
4                       sir hero chance remember thats
Name: comment_text, dtype: object

In [33]:
# Applying rare word edit code to 'test' dataframe
freq = pd.Series(' '.join(test['comment_text']).split()).value_counts()[-10:]
freq = list(freq.index)
test['comment_text'] = test['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
test['comment_text'].head()

0    yo bitch ja rule succesful youll ever whats ha...
1                                   rfc title fine imo
2                          sources zawe ashton lapland
3    look back source information updated correct f...
4                            anonymously edit articles
Name: comment_text, dtype: object

### Lemmatization

In [34]:
### LEMMATIZATION ###

# Importing 'Word' library
from textblob import Word
# Lemmatizing comments so only lemmas remain
train['comment_text'] = train['comment_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
# Sample of 'train' dataframe 'comment_text' column with lemmatization
train['comment_text'].head()

0    explanation edits made username hardcore metal...
1    daww match background colour im seemingly stuc...
2    hey man im really trying edit war guy constant...
3    cant make real suggestion improvement wondered...
4                       sir hero chance remember thats
Name: comment_text, dtype: object

In [35]:
# Applying rare word edit code to 'test' dataframe
test['comment_text'] = test['comment_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

# Extracting features

### TF-IDF matrix

In [53]:
## EXTRACTING FEATURES ##
### TF-IDF MATRIX ###

# Reading in test_labels.csv into variable 'test_labels'
test_labels = pd.read_csv('C:/Users/Admin/OneDrive/Spring 19 Classes/STA 141C/Project/jigsaw-toxic-comment-classification-challenge/test_labels.csv')

In [62]:
# Subsetting 'test_labels' dataframe with rows containing -1 in all toxicity classification labels and saving to variable 'test_neg1'
test_neg1 = test_labels.loc[(test_labels["toxic"] == -1) & (test_labels["severe_toxic"] == -1) & 
                           (test_labels["obscene"] == -1) & (test_labels["threat"] == -1) & 
                           (test_labels["insult"] == -1) & (test_labels["identity_hate"] == -1)]
# Creating list of indices of rows from 'test_label' dataframe that contain -1 in all toxicity classification labels and saving to variable 'index_neg1'
index_neg1 = list(test_neg1.index.values.astype(int))
# Dropping rows from 'test_label' dataframe that match index numbers from 'index_neg1' and saving new dataframe to variable 'noneg_test_labels'
noneg_test_labels = test_labels.drop(index_neg1)
# Dropping rows from 'test' dataframe that match index numbers from 'index_neg1' and saving new dataframe to variable 'noneg_test'
noneg_test = test.drop(index_neg1)
# Sample of 'noneg_test' dataframe after dropping rows
noneg_test.head()

,id,comment_text,word_count,char_count,upper
5,0001ea8717f6de06,thank understanding highly revert without disc...,16,96,1
7,000247e83dcc1211,dear god site horrible,6,32,0
11,0002f87b16116a7f,somebody invariably try add religion really me...,78,448,0
13,0003e1cccfd5a40a,say right type type institution needed case th...,90,501,5
14,00059ace3e3e9a53,adding new product list make sure relevant add...,60,334,0


In [63]:
# Combining text from comments from 'train' dataframe and 'noneg_test' dataframe into one series 
all_text = pd.concat([train['comment_text'], noneg_test['comment_text']])

In [64]:
# Setting up TF-IDF vectorizer and saving to variable 'tfidf'
tfidf = TfidfVectorizer(max_features=1000, lowercase = True, analyzer = 'word',
                       stop_words = 'english', ngram_range=(1,1), strip_accents = 'unicode')
# Applying TF-IDF vectorizer to 'all_text'
tfidf.fit(all_text)
# Transforming on comments from 'train' document and 'noneg_test' document and saving to respective variables 'train_feautres' and 'test_features'
train_features = tfidf.transform(train['comment_text'])
test_features = tfidf.transform(noneg_test['comment_text'])

In [65]:
# Result of transformation with 'train_features'
train_features

<159571x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 2022615 stored elements in Compressed Sparse Row format>

In [66]:
# Result of transformation with 'test_features'
test_features

<63978x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 738672 stored elements in Compressed Sparse Row format>

In [71]:
# Creating list of toxicity classification labels and saving to variable 'class_names' 
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [77]:
# Applying cross-validation to 'train_features' and calculating cross-validation scores for each category
scores = []
for i in class_names:
    cl = train[i]
    classifier = LogisticRegression(C = 0.1, solver = 'sag')
    train_score = np.mean(cross_val_score(classifier, train_features, cl, cv=3, scoring = 'roc_auc'))
    scores.append(train_score)                                     

In [85]:
scores

list

In [87]:
# Converting variable 'scores' to dataframe
scores = pd.DataFrame(scores)

In [88]:
# Calculating mean of 'scores'
scores.mean()

0    0.956469
dtype: float64

In [51]:
train['toxic']

0         0
1         0
2         0
3         0
4         0
5         0
6         1
7         0
8         0
9         0
10        0
11        0
12        1
13        0
14        0
15        0
16        1
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
159541    1
159542    0
159543    0
159544    0
159545    0
159546    1
159547    0
159548    0
159549    0
159550    0
159551    0
159552    0
159553    0
159554    1
159555    0
159556    0
159557    0
159558    0
159559    0
159560    0
159561    0
159562    0
159563    0
159564    0
159565    0
159566    0
159567    0
159568    0
159569    0
159570    0
Name: toxic, Length: 159571, dtype: int64

In [52]:
# Applying cross-validation to toxic column of 'train' and calculating cross-validation score
toxic_train_score = np.mean(cross_val_score(classifier, train_features, train['toxic'], cv=3, scoring = 'roc_auc'))
toxic_train_score

0.9361748580251202

In [58]:
# Sample of 'test_labels' dataframe
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [96]:
# Applying cross-validation to 'test_features' and calculating cross-validation score
for i in class_names:
    cl = noneg_test_labels[i]
    test_score = np.mean(cross_val_score(classifier, test_features, cl, cv=3, scoring = 'roc_auc'))
    test_scores = scores.append(test_score)  

In [97]:
# List of test scores from cross-validation; score for each toxicity classification label
test_scores

[0.9158625616784715,
 0.9660691521596756,
 0.9290434781128654,
 0.9700867231176814,
 0.9292141198381404,
 0.9559217260945632]

In [99]:
# Converting 'test_scores' into dataframe and calculating mean of scores
test_scores = pd.DataFrame(scores)
test_scores.mean()

0    0.944366
dtype: float64

multi-label classification
screen command for remote work 

neural networks